## Version History:
### Latest Updated by Bryant on 2025/5/5 16:45

### Progress:
1. 輸出的 optimility gap 都是正的

### Code Updates:
1. [J = 3  # NEW: added by Byrant](https://github.com/Bryant-Liu-TzuYang/OR-Team-Cospace/blob/8cae18753580a1c3147701cd6be101c5f8fe1224/MidtermProjectExperiment_new.ipynb)



## Example

In [21]:
import pandas as pd
import numpy as np

######################## PARAMTERS STARTING HERE ################################
# Read the Excel file from the 'Demand' sheet
file_path = "OR113-2_midtermProject_data.xlsx"
df_demand = pd.read_excel(file_path, sheet_name="Demand")
N = df_demand.shape[0] - 1   # -1 because of the first row, +1 for indices' consistency
T = df_demand.shape[1] - 2  # -2 because of the first two columns, +1 for indices' consistency  
print("N:", N, "T:", T)

# Display the dataframe to verify the data
I = np.zeros([N, T])
D = np.zeros([N, T])
I_0 = np.zeros([N])

for i in range(N):
    I_0[i] = df_demand.iloc[i+1, 1]
    for t in range(T):
        D[i, t] = df_demand.iloc[i+1, t+2]

print("I_0:", I_0)
print("D:", D)

# Read the Excel file from the 'In-transit' sheet
df_in_transit = pd.read_excel(file_path, sheet_name="In-transit")
for i in range(N):
    for t in range(df_in_transit.shape[1] - 1):
        I[i, t] = df_in_transit.iloc[i+1, t+1]
print("I:", I)

# Read the Excel file from the 'Shipping cost' sheet
df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost")
print(df_shipping_cost.shape)
J = df_shipping_cost.shape[1] - 1 # -1 because of the first column
df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")


C = {
    "H": np.zeros([N]),
    "P": np.zeros([N]),
    "V": np.zeros([N, J]),
    "F": np.array([100, 80, 50]),
    "C": 2750,
}
V = np.zeros([N])
V_C = 30
P = np.zeros([N])
BP_percentage = np.zeros([N])
for i in range(N):
    P[i] = df_inventory_cost.iloc[i, 1]
    BP_percentage[i] = df_inventory_cost.iloc[i, 5]
    C["H"][i] = df_inventory_cost.iloc[i, 3]
    C["P"][i] = df_inventory_cost.iloc[i, 2]
    V[i] = df_shipping_cost.iloc[i, 3]
    for j in range(J):
        if j == J - 1:
            C["V"][i, j] = 0
        else:
            C["V"][i, j] = df_shipping_cost.iloc[i, j+1]

print("P:", P)
print("BP_percentage:", BP_percentage)
print("C:", C)
print("V:", V)
print("J:", J)
T_lead = np.array([1, 2, 3]) # T_j

N: 10 T: 6
I_0: [800. 600. 425. 350. 400. 524. 453. 218. 673. 200.]
D: [[138.  55. 172. 194.  94. 185.]
 [190. 101.  68. 185.  13. 136.]
 [ 79. 179.  21.  49. 199. 200.]
 [142. 103.  78. 131. 146. 155.]
 [ 35.  62.  83.  90. 197.  49.]
 [ 91.  95. 107. 127. 116. 183.]
 [105. 164.  19. 116. 119. 175.]
 [ 37. 155.  10.  77. 168.  32.]
 [108. 185. 188. 176.  81. 172.]
 [ 46. 178. 162. 200. 154. 199.]]
I: [[  0.   0.   0.   0.   0.   0.]
 [ 48.   0.   0.   0.   0.   0.]
 [  0.  20.   0.   0.   0.   0.]
 [153.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [ 18.  23.   0.   0.   0.   0.]
 [ 28.  45.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]
 [109.  34.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.]]
(10, 9)
P: [10700.  5000. 26900. 19600.  6900. 31500. 14100. 16400. 33000. 18600.]
BP_percentage: [0.  0.7 0.1 1.  1.  0.3 0.6 0.2 0.1 0.5]
C: {'H': array([100.,  40., 180., 180.,  40., 180., 140., 100., 180., 140.]), 'P': array([5000., 2000., 9000., 9000., 2000.,

In [18]:
# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

for i in S_I:
    print(f"i = {i}")

for t in S_T:
    print(f"t = {t}")

for j in S_J:
    print(f"j = {j}")



i = 0
i = 1
i = 2
i = 3
i = 4
i = 5
i = 6
i = 7
i = 8
i = 9
t = 0
t = 1
t = 2
t = 3
t = 4
t = 5
j = 0
j = 1
j = 2
j = 3
j = 4
j = 5
j = 6
j = 7


In [19]:
import gurobipy as gp
from gurobipy import GRB

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t

# Objective function (1)
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for j in S_J for t in S_T)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

# Constraints
# Inventory balance (2)
J_in_inventory = np.array([1, 2, 3, 3, 3, 3])

for i in S_I:
    for t in S_T:
        # Compute the in-transit quantity arriving at time t
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            in_inventory += x[i, j, t - T_lead[j] + 1]
        # Add the constraint for inventory balance
        if t == 0:
            model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
        else:
            model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

# Relate order quantity and shipping method (4)
M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
for j in S_J:
    for t in S_T:
        model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

# Container constraint (5)
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# Non-negativity and binary constraints (6)
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for i in S_I:
    for t in S_T:
        model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()

Set parameter MIPGap to value 0


IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.objVal)
    print("\nOrder quantities (x_ijt):")
    
    for t in S_T:
        for i in S_I:
            for j in S_J:
                    if x[i, j, t].x > 0:
                        print(f"x[{i+1},{j+1},{t+1}] = {x[i, j, t].x}") # +1 to make the index consistent
    print("\nEnding inventory (v_it):")
    for t in S_T:
        for i in S_I:
                if v[i, t].x > 0:
                    print(f"v[{i+1},{t+1}] = {v[i, t].x}")
    print("\nShipping method usage (y_jt):")
    for t in S_T:
        for j in S_J:
                if y[j, t].x > 0:
                    print(f"y[{j+1},{t+1}] = {y[j, t].x}")
    print("\nNumber of containers (z_t):")
    for t in S_T:
        if z[t].x > 0:
            print(f"z[{t+1}] = {z[t].x}")
else:
    print("No optimal solution found.")


Optimal objective value: 16890367.020408165

Order quantities (x_ijt):
x[8,3,1] = 61.0
x[10,1,1] = 24.0
x[10,2,1] = 162.0
x[10,3,1] = 200.0
x[3,3,2] = 82.0
x[4,3,2] = 97.0
x[5,3,2] = 67.0
x[8,3,2] = 168.0
x[10,3,2] = 36.44897959183673
x[1,3,3] = 38.0
x[2,3,3] = 45.0
x[3,3,3] = 200.0
x[4,3,3] = 155.0
x[5,3,3] = 49.0
x[6,3,3] = 154.0
x[7,3,3] = 172.0
x[8,3,3] = 32.0
x[9,3,3] = 94.0
x[10,2,3] = 117.55102040816327
x[10,3,3] = 199.0

Ending inventory (v_it):
v[1,1] = 662.0
v[2,1] = 458.0
v[3,1] = 346.0
v[4,1] = 361.0
v[5,1] = 365.0
v[6,1] = 451.0
v[7,1] = 376.0
v[8,1] = 181.0
v[9,1] = 674.0
v[10,1] = 178.0
v[1,2] = 607.0
v[2,2] = 357.0
v[3,2] = 187.0
v[4,2] = 258.0
v[5,2] = 303.0
v[6,2] = 379.0
v[7,2] = 257.0
v[8,2] = 26.0
v[9,2] = 523.0
v[10,2] = 162.0
v[1,3] = 435.0
v[2,3] = 289.0
v[3,3] = 166.0
v[4,3] = 180.0
v[5,3] = 220.0
v[6,3] = 272.0
v[7,3] = 238.0
v[8,3] = 77.0
v[9,3] = 335.0
v[10,3] = 200.0
v[1,4] = 241.0
v[2,4] = 104.0
v[3,4] = 199.0
v[4,4] = 146.0
v[5,4] = 197.0
v[6,4] = 145.0


In [ ]:
#original problem, only adjust the period number so that it's unconstrained
import gurobipy as gp
from gurobipy import GRB

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

# Create the Gurobi model
model = gp.Model("InventoryManagement")

# Set error parameter
model.setParam('MIPGap', 0.0)

# Define sets
S_I = range(N)  # Products i in {0,  ..., N-1}
S_T = range(T)  # Time periods t in {0, ..., T-1}
S_J = range(J)  # Shipping methods j in {0, ..., J-1}

# Variables
x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
y = model.addVars(S_J, S_T, vtype=GRB.BINARY, name="y")  # Binary for shipping method y_jt
z = model.addVars(S_T, vtype=GRB.INTEGER, name="z")  # Number of containers z_t

# Objective function (1)
# Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
purchasing_and_shipping_cost = gp.quicksum(
    (C["P"][i] + C["V"][i, j]) * x[i, j, t]
    for i in S_I for j in S_J for t in S_T
) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

# Constraints
# Inventory balance (2)
J_in_inventory = list()

for j in S_T:
    if j == 0:
        J_in_inventory.append(1)
    elif j == 1:
        J_in_inventory.append(2)
    else:
        J_in_inventory.append(3)

J_in_inventory = np.array(J_in_inventory)

for i in S_I:
    for t in S_T:
        # Compute the in-transit quantity arriving at time t
        in_inventory = 0
        for j in range(J_in_inventory[t]):
            in_inventory += x[i, j, t - T_lead[j] + 1]
        # Add the constraint for inventory balance
        if t == 0:
            model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
        else:
            model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

# Relate order quantity and shipping method (4)
M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
for j in S_J:
    for t in S_T:
        model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

# Container constraint (5)
for t in S_T:
    model.addConstr(
        gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
        name=f"Container_{t}"
    )

# Non-negativity and binary constraints (6)
for i in S_I:
    for j in S_J:
        for t in S_T:
            model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
for i in S_I:
    for t in S_T:
        model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
for j in S_J:
    for t in S_T:
        model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
for t in S_T:
    model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

# Optimize the model
model.optimize()

Set parameter MIPGap to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
MIPGap  0

Optimize a model with 398 rows, 264 columns and 838 nonzeros
Model fingerprint: 0xbe75289a
Variable types: 240 continuous, 24 integer (18 binary)
Coefficient statistics:
  Matrix range     [5e-03, 7e+03]
  Objective range  [4e+01, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+02]
Found heuristic solution: objective 1.967757e+07
Presolve removed 365 rows and 122 columns
Presolve time: 0.00s
Presolved: 33 rows, 142 columns, 300 nonzeros
Found heuristic solution: objective 1.697771e+07
Variable types: 128 continuous, 14 integer (11 binary)

Root relaxation: objective 1.688731e+07, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node  

## Solve Linear relaxation

In [26]:
#linear relaxation
#also adjust the period number so that it's unconstrained
import gurobipy as gp
from gurobipy import GRB


def LR(N,T,J,C,I,I_0,D,V,V_C):
    # Provided parameters (already read from the Excel file)
    # N: number of products, T: number of time periods, J: number of shipping methods
    # D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

    # Create the Gurobi model
    model = gp.Model("InventoryManagement")

    # Set error parameter
    model.setParam('MIPGap', 0.0)

    # Define sets
    S_I = range(N)  # Products i in {0,  ..., N-1}
    S_T = range(T)  # Time periods t in {0, ..., T-1}
    S_J = range(J)  # Shipping methods j in {0, ..., J-1}

    # Variables
    x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
    v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
    y = model.addVars(S_J, S_T, vtype=GRB.CONTINUOUS, name="y")  # Relaxed: Binary for shipping method y_jt
    z = model.addVars(S_T, vtype=GRB.CONTINUOUS, name="z")  # Relaxed: Number of containers z_t

    # Objective function (1)
    # Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
    holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
    purchasing_and_shipping_cost = gp.quicksum(
        (C["P"][i] + C["V"][i, j]) * x[i, j, t]
        for i in S_I for j in S_J for t in S_T
    ) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
    container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

    model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

    # Constraints
    # Inventory balance (2)
    J_in_inventory = list()

    for j in S_T:
        if j == 0:
            J_in_inventory.append(1)
        elif j == 1:
            J_in_inventory.append(2)
        else:
            J_in_inventory.append(3)

    for i in S_I:
        for t in S_T:
            # Compute the in-transit quantity arriving at time t
            in_inventory = 0
            for j in range(J_in_inventory[t]):
                in_inventory += x[i, j, t - T_lead[j] + 1]
            # Add the constraint for inventory balance
            if t == 0:
                model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            else:
                model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
                model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

    # Relate order quantity and shipping method (4)
    M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
    for j in S_J:
        for t in S_T:
            model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

    # Container constraint (5)
    for t in S_T:
        model.addConstr(
            gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
            name=f"Container_{t}"
        )

    # Non-negativity and binary constraints (6)
    for i in S_I:
        for j in S_J:
            for t in S_T:
                model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
    for i in S_I:
        for t in S_T:
            model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
    for j in S_J:
        for t in S_T:
            model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
            model.addConstr(y[j, t] <= 1, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
    for t in S_T:
        model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

    # Optimize the model
    model.optimize()
    return(model.objVal)

## Solve Naive

In [27]:
#linear relaxation
#use express only, always order the number we need in next month
import gurobipy as gp
from gurobipy import GRB

# Provided parameters (already read from the Excel file)
# N: number of products, T: number of time periods, J: number of shipping methods
# D: demand, I: in-transit inventory, C: cost parameters, V: volume, T_lead: lead times, V_C: container volume

def naive(N,T,J,C,I,I_0,D,V,V_C):
    # Create the Gurobi model
    model = gp.Model("InventoryManagement")

    # Set error parameter
    model.setParam('MIPGap', 0.0)

    # Define sets
    S_I = range(N)  # Products i in {0,  ..., N-1}
    S_T = range(T)  # Time periods t in {0, ..., T-1}
    S_J = range(J)  # Shipping methods j in {0, ..., J-1}

    # Variables
    x = model.addVars(S_I, S_J, S_T, vtype=GRB.CONTINUOUS, name="x")  # Order quantity x_ijt
    v = model.addVars(S_I, S_T, vtype=GRB.CONTINUOUS, name="v")  # Ending inventory v_it
    y = model.addVars(S_J, S_T, vtype=GRB.CONTINUOUS, name="y")  # Relaxed: Binary for shipping method y_jt
    z = model.addVars(S_T, vtype=GRB.CONTINUOUS, name="z")  # Relaxed: Number of containers z_t

    # Objective function (1)
    # Holding cost + (Purchasing cost + Variable shipping cost + Fixed shipping cost) + Container cost
    holding_cost = gp.quicksum(C["H"][i] * v[i, t] for i in S_I for t in S_T)
    purchasing_and_shipping_cost = gp.quicksum(
        (C["P"][i] + C["V"][i, j]) * x[i, j, t]
        for i in S_I for j in S_J for t in S_T
    ) + gp.quicksum(C["F"][j] * y[j, t] for t in S_T for j in S_J)
    container_cost = gp.quicksum(C["C"] * z[t] for t in S_T)

    model.setObjective(holding_cost + purchasing_and_shipping_cost + container_cost, GRB.MINIMIZE)

    # Constraints

    #new constraints: each time we order for exactly next month's demand, use express only, ignore product in transit and initial inventory
    for i in S_I:
        for t in S_T:
            if t == len(S_T) - 1:
                model.addConstr(x[i, 0, t] == 0, name=f"last month no order")
            else:
                model.addConstr(x[i, 0, t] == D[i,t+1], name=f"order for next month demand")

            model.addConstr(x[i, 1, t] == 0, name=f"No air freight")
            model.addConstr(x[i, 2, t] == 0, name=f"No ocean freight")


    # Inventory balance (2)
    J_in_inventory = list()

    for j in S_T:
        if j == 0:
            J_in_inventory.append(1)
        elif j == 1:
            J_in_inventory.append(2)
        else:
            J_in_inventory.append(3)

    for i in S_I:
        for t in S_T:
            # Compute the in-transit quantity arriving at time t
            in_inventory = 0
            for j in range(J_in_inventory[t]):
                in_inventory += x[i, j, t - T_lead[j] + 1]
            # Add the constraint for inventory balance
            if t == 0:
                model.addConstr(v[i, t] == in_inventory + I_0[i] + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
            else:
                model.addConstr(v[i, t] == v[i, t-1] + in_inventory + I[i, t] - D[i, t], name=f"InvBalance_{i}_{t}")
                model.addConstr(v[i, t-1] >= D[i, t], name=f"Demand_{i}_{t}")

    # Relate order quantity and shipping method (4)
    M = sum(sum(D[i, t] for t in S_T) for i in S_I)  # Large number M as per problem statement
    for j in S_J:
        for t in S_T:
            model.addConstr(gp.quicksum(x[i, j, t] for i in S_I) <= M * y[j, t], name=f"ShippingMethod_{j}_{t}")

    # Container constraint (5)
    for t in S_T:
        model.addConstr(
            gp.quicksum(V[i] * x[i, 2, t] for i in S_I) <= V_C * z[t],
            name=f"Container_{t}"
        )

    # Non-negativity and binary constraints (6)
    for i in S_I:
        for j in S_J:
            for t in S_T:
                model.addConstr(x[i, j, t] >= 0, name=f"NonNeg_x_{i}_{j}_{t}")
    for i in S_I:
        for t in S_T:
            model.addConstr(v[i, t] >= 0, name=f"NonNeg_v_{i}_{t}")
    for j in S_J:
        for t in S_T:
            model.addConstr(y[j, t] >= 0, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
            model.addConstr(y[j, t] <= 1, name=f"Binary_y_{j}_{t}")  # Already binary due to vtype
    for t in S_T:
        model.addConstr(z[t] >= 0, name=f"NonNeg_z_{t}")

    # Optimize the model
    model.optimize()
    return(model.objVal)

In [28]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal objective value:", model.objVal)
    print("\nOrder quantities (x_ijt):")
    
    for t in S_T:
        for i in S_I:
            for j in S_J:
                    if x[i, j, t].x > 0:
                        print(f"x[{i+1},{j+1},{t+1}] = {x[i, j, t].x}") # +1 to make the index consistent
    print("\nEnding inventory (v_it):")
    for t in S_T:
        for i in S_I:
                if v[i, t].x > 0:
                    print(f"v[{i+1},{t+1}] = {v[i, t].x}")
    print("\nShipping method usage (y_jt):")
    for t in S_T:
        for j in S_J:
                if y[j, t].x > 0:
                    print(f"y[{j+1},{t+1}] = {y[j, t].x}")
    print("\nNumber of containers (z_t):")
    for t in S_T:
        if z[t].x > 0:
            print(f"z[{t+1}] = {z[t].x}")
else:
    print("No optimal solution found.")

No optimal solution found.


## Solve heuristic

In [29]:
#Main heuristic

import numpy as np

def heuristic_shipping_optimizer(N, T, C, in_transit, inventory,  forecast, V, container_capacity):
    T_lead = [1, 2, 3]
    plan = {0: np.zeros((N, T)), 1: np.zeros((N, T)), 2: np.zeros((N, T))}
    shortages = np.zeros((N, T))
    leftover_volumes = np.zeros(T)
    full_containers = np.zeros(T)
    already_aggregated = np.zeros((N, T))
    leftover_product = np.zeros((N, T))

    holding_costs = C["H"]
    shipping_fixed = C["F"]
    shipping_variable = C["V"]
    container_cost = C["C"]
    purchasing_cost = C["P"]

    # Step 1: Calculate shortages (use only initial inventory + in_transit)
    I_remain = np.zeros((N, T + 1))
    for i in range(N):
        I_remain[i, 0] = inventory[i]  # 修正為一維初始庫存

    for t in range(T):
        for i in range(N):
            demand = forecast[i, t]

            available = I_remain[i, t]
            shortage = max(0, demand - available)
            shortages[i, t] = shortage

            next_inventory = max(0, available - demand) + in_transit[i, t]
            I_remain[i, t + 1] = next_inventory

    # Handle early-month shortages
    for t in range(T):
        for i in range(N):
            if shortages[i, t] > 0:
                if t - T_lead[1] < 0 and t - T_lead[2] < 0:
                    if t - T_lead[0] >= 0:
                        qty = shortages[i, t]
                        plan[0][i, t - T_lead[0]] += qty
                        shortages[i, t] = 0
                elif t - T_lead[1] >= 0 and t - T_lead[2] < 0:
                    qty = shortages[i, t]
                    plan[1][i, t - T_lead[1]] += qty
                    shortages[i, t] = 0

    # Step 2: Allocate full containers using ocean
    for t in range(T):
        air_cost_per_m3 = [(i, shipping_variable[i, 1] / V[i]) for i in range(N) if shortages[i, t] > 0 and t - T_lead[2] >= 0]
        air_cost_per_m3.sort(key=lambda x: x[1], reverse=True)

        vol_accum = sum(shortages[i, t] * V[i] for i, _ in air_cost_per_m3)
        full = np.floor(vol_accum / container_capacity)
        leftover_volume = vol_accum - full * container_capacity
        leftover_volumes[t] = leftover_volume
        full_containers[t] = full

        vol_used = 0
        for i, _ in air_cost_per_m3:
            qty = shortages[i, t]
            vol = qty * V[i]
            
            if full == 0:
                # 🔧 完全不足一櫃，保留到後面 Step 4 合併
                leftover_product[i, t] = qty
                continue

            available_vol = full * container_capacity - vol_used
            if available_vol <= 0:
                break

            if vol <= available_vol:
                plan[2][i, t - T_lead[2]] += qty
                shortages[i, t] = 0
                vol_used += vol
            else:
                qty_fit = available_vol // V[i]
                if qty_fit > 0:
                    plan[2][i, t - T_lead[2]] += qty_fit
                    shortages[i, t] -= qty_fit
                    leftover_product[i, t] = shortages[i, t]
                    vol_used += qty_fit * V[i]

    # Step 3: Aggregate leftover if beneficial
    for t in range(T - 1):
        if leftover_volumes[t] > 0 and t - T_lead[2] >= 0:
            combined_vol = leftover_volumes[t] + leftover_volumes[t + 1]
            if combined_vol <= container_capacity:
                saved_cost = shipping_fixed[2] + container_cost
                holding_cost = sum(holding_costs[i] * leftover_product[i, t + 1] for i in range(N))

                aggregate_qty = [leftover_product[i, t] + leftover_product[i, t + 1] for i in range(N)]
                ocean_cost = shipping_fixed[2] + container_cost
                air_cost = sum(aggregate_qty[i] *  shipping_variable[i, 1] for i in range(N)) + shipping_fixed[1]

                if saved_cost > holding_cost and ocean_cost > air_cost:
                    for i in range(N):
                        leftover_product[i, t] += leftover_product[i, t + 1]
                        plan[2][i, t - T_lead[2]] += leftover_product[i, t]
                        leftover_product[i, t + 1] = 0
                    leftover_volumes[t] = combined_vol
                    leftover_volumes[t + 1] = 0
                    already_aggregated[:, t] = 1
                    full_containers[t] += 1  # aggregated container

    # Step 4: Decide how to transport leftover volumes
    for t in range(T):
        if np.all(already_aggregated[:, t] == 0) and leftover_volumes[t] > 0 and t - T_lead[2] >= 0:
            vol = leftover_volumes[t]
            containers_needed = np.ceil(vol / container_capacity)

            ocean_cost = shipping_fixed[2] + containers_needed * container_cost
            air_cost = shipping_fixed[1] + sum(shipping_variable[i, 1] * leftover_product[i, t] for i in range(N))

            method = 1 if air_cost < ocean_cost else 2
            for i in range(N):
                qty = leftover_product[i, t]
                if qty > 0 and t - T_lead[method] >= 0:
                    plan[method][i, t - T_lead[method]] += qty
                    shortages[i, t] = 0
            leftover_volumes[t] = 0

            if method == 2:
                full_containers[t] += 1  # unaggregated ocean shipment

    # Step 5: Calculate total cost
    stock = np.zeros((N, T + 1))  # [產品 i][月 t]，多一格存最後庫存
    forecast_copy = forecast.copy()
    total_holding_cost = 0

    # Step 1: 初始庫存設定（第0月代表月初可用）
    stock[:, 0] = np.array(inventory)  # inventory 是初始庫存，在月1可用

    # Step 2: 把所有訂單的到貨時間加到對應月的庫存
    for k in range(3):  # 三種運輸方式
        for i in range(N):
            for t in range(T):
                arrival_month = t + T_lead[k]
                if arrival_month < T:
                    stock[i, arrival_month] += plan[k][i, t]
                    # ➕ 提前一個月把這批貨的 holding cost 算進去（即到達月 - 1）
                    if arrival_month - 1 >= 0:
                        total_holding_cost += plan[k][i, t] * holding_costs[i]

    # Step 3: 逐月計算 holding cost 和更新庫存
    for t in range(T):
        for i in range(N):
            # Step 3.1: 當月可用庫存（不含當月 in_transit）
            available = stock[i, t]
            used = min(available, forecast_copy[i, t])
            forecast_copy[i, t] -= used
            leftover = available - used

            # Step 3.2: 本月 holding cost（只算不能出貨的貨：剩下的 + in_transit）
            total_holding_cost += (leftover + in_transit[i, t]) * holding_costs[i]

            # Step 3.3: 更新下一月庫存（剩下的貨 + 當月的 in_transit 貨）
            if t + 1 <= T:
                stock[i, t + 1] += leftover + in_transit[i, t]


    total_shipping_cost = 0
    total_purchasing_cost = 0
    total_container_cost = 0
    shipping_methods_used = {0: set(), 1: set(), 2: set()}

    # Express & Air: calculate purchasing and shipping
    for k in [0, 1]:
        for t in range(T):
            method_volume = sum(plan[k][i, t] for i in range(N))
            if method_volume > 0:
                shipping_methods_used[k].add(t)
            for i in range(N):
                qty = plan[k][i, t]
                total_purchasing_cost += qty * purchasing_cost[i]
                total_shipping_cost += qty * shipping_variable[i, k]

    # Ocean: only purchasing cost（
    for t in range(T):
        method_volume = sum(plan[2][i, t] for i in range(N)) 
        if method_volume > 0:
            shipping_methods_used[2].add(t)
        for i in range(N):
            qty = plan[2][i, t]
            total_purchasing_cost += qty * purchasing_cost[i]

    # Fixed cost for all shipping modes
    for k in range(3):
        total_shipping_cost += len(shipping_methods_used[k]) * shipping_fixed[k]

    # Ocean container cost
    total_container_cost += np.sum(full_containers) * container_cost

    total_cost = total_holding_cost + total_shipping_cost + total_purchasing_cost + total_container_cost

    return total_cost


## Sampling

In [30]:
import random

a = random.uniform(10, 20) # [10, 20]
a

17.64049723717511

| Scale | Number products/Periods | Container | Holding cost |
|--|---------------------------------|-----------|--------------|
| small |   10/6                              |    1375       |        1%      |
| medium |      100/20                           |      2750     |       2%       |
| large |     500/50           |       5500    |       4%       | 

In [31]:
scale_dict = {
    "small":{"N": 10, "T": 6, "C_C": 1375, "C_H": 0.01},
    "medium":{"N": 100, "T": 20, "C_C": 2750, "C_H": 0.02},
    "large":{"N": 500, "T": 50, "C_C": 5500, "C_H": 0.04}
}

#low -> small
#high -> large
 
scenerio_dict = {
    1: {"scale": "medium", "C_C": "medium", "C_H": "medium"},
    2: {"scale": "small", "C_C": "medium", "C_H": "medium"},
    3: {"scale": "large", "C_C": "medium", "C_H": "medium"},
    4: {"scale": "medium", "C_C": "small", "C_H": "medium"},
    5: {"scale": "medium", "C_C": "large", "C_H": "medium"},
    6: {"scale": "medium", "C_C": "medium", "C_H": "small"},
    7: {"scale": "medium", "C_C": "medium", "C_H": "large"}
}

In [33]:
file_path = "OR113-2_midtermProject_data.xlsx"

import time
import random
import numpy as np
import pandas as pd

LR_obj_list = list()
naive_obj_list = list()
heuristic_obj_list = list()
naive_optimality_gap = list()
heuristic_optimality_gap = list()
LR_computation_time_list = list()
naive_computation_time_list = list()
heuristic_computation_time_list = list()

## 7 scenarios
random.seed(42)
np.random.seed(42)
for scenerio in range(1,8): #(1,8)
    #for each scenario, generate 30 instances
    for instance in range(1, 31): #(1, 31)
        
        scaleof = scenerio_dict[scenerio]
        
        N = scale_dict[scaleof["scale"]]["N"]
        T = scale_dict[scaleof["scale"]]["T"]
        J = 3  # NEW: added by Byrant
        #print("N:", N, "T:", T) #print product number, period number in this scenario
        

        I = np.zeros([N, T])
        D = np.zeros([N, T])
        I_0 = np.zeros([N])
        
        for i in range(N):
            for t in range(T):
                D[i, t] = random.uniform(0, 200)
    
            I_0[i] = random.uniform(D[i, 0], 400)
        
        #print("I_0:", I_0) #initial inventory of i, not smaller than D[i,0]
        #print("D:", D) #demand D[i,t]
        
        
        # Read the Excel file from the 'In-transit' sheet ##it's needed??
        #df_in_transit = pd.read_excel(file_path, sheet_name="In-transit") 

        for i in range(N):
            for t in range(T):
                if t == 0: # first period
                    I[i, 0] = random.choice([0, 1]) * random.uniform(0, 200) # correspond to I_i,1
                elif t == 1: # second period
                    I[i, 0] = random.choice([0, 1]) * random.uniform(0, 50) # correspond to I_i,2                       
                else:
                    I[i, t] = 0
        
        #print("I:", I) #in transit
        
        
        # Read the Excel file from the 'Shipping cost' sheet ##it's needed??
        #df_shipping_cost = pd.read_excel(file_path, sheet_name="Shipping cost") 
        #J = df_shipping_cost.shape[1] - 1 # -1 because of the first column
        #df_inventory_cost = pd.read_excel(file_path, sheet_name="Inventory cost")
        
        #all costs
        C = {
            "H": np.zeros([N]),
            "P": np.zeros([N]),
            "V": np.zeros([N, J]),
            "F": np.array([100, 80, 50]), #fixed
            "C": scale_dict[scaleof["C_C"]]["C_C"], ##找到scenario對應的container cost (originally 2750)
        }

        
        V = np.zeros([N])#vi: volume of product i
        V_C = 30
        for i in range(N):
            C["P"][i] = random.uniform(1000, 10000)
            C["H"][i] = C["P"][i] * scale_dict[scaleof["C_H"]]["C_H"] ##找到scenario對應的holding cost比例 
            
            V[i] = random.uniform(0, 1)

            #variable cost of two shipping method 
            C["V"][i, 0] = 0
            C["V"][i, 1] = random.uniform(40, 100)
            C["V"][i, 2] = random.uniform(0.4, 0.6) # No variable shipping cost for ocrean freight
        
        #print("C:", C)
        #print("V:", V)
        T_lead = np.array([1, 2, 3]) # T_j, lead time of 3 shipping methods
    
        start_time = time.time()
        LR_obj = LR(N,T,J,C,I,I_0,D,V,V_C)
        end_time = time.time()
        computation_time = end_time - start_time
        LR_computation_time_list.append(computation_time)

        LR_obj_list.append(LR_obj)

        start_time = time.time()
        naive_obj = naive(N,T,J,C,I,I_0,D,V,V_C)
        end_time = time.time()
        computation_time = end_time - start_time
        naive_computation_time_list.append(computation_time)
        naive_obj_list.append(naive_obj)

        start_time = time.time()
        heuristic_obj= heuristic_shipping_optimizer(N,T,C,I,I_0,D,V,V_C)
        heuristic_obj_list.append(heuristic_obj)
        end_time = time.time()
        computation_time = end_time - start_time
        heuristic_computation_time_list.append(computation_time)

        naive_optimality_gap.append((naive_obj - LR_obj) / LR_obj)
        heuristic_optimality_gap.append((heuristic_obj - LR_obj) / LR_obj)

Set parameter MIPGap to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Non-default parameters:
MIPGap  0

Optimize a model with 12120 rows, 8080 columns and 27720 nonzeros
Model fingerprint: 0xb3493fff
Coefficient statistics:
  Matrix range     [1e-02, 2e+05]
  Objective range  [2e+01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-02, 4e+02]
Presolve removed 10327 rows and 887 columns
Presolve time: 0.01s
Presolved: 1793 rows, 7193 columns, 14232 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.2322629e+07   2.194448e+04   0.000000e+00      0s
    3207    9.1043732e+08   0.000000e+00   0.000000e+00      0s

Solved in 3207 iterations and 0.04 seconds (0.02 work units)
Optimal objective  9.104373202e+08
Set parameter MIPGap to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (

In [34]:
print(LR_obj_list)

[910437320.1712431, 958556275.0807927, 973547649.972581, 992092652.3887901, 872325306.8450257, 1033062887.1174656, 906883480.041765, 924327709.8927268, 973968229.2653648, 970524695.1562628, 871660390.8564374, 916367528.3492403, 898879502.5568131, 1019228780.949725, 936327174.5646181, 983353764.3769772, 1011011825.0223536, 953416583.8697841, 945164690.9590337, 964540793.2627566, 1092192927.0359178, 1053415671.4640567, 974269268.031586, 1011625062.1593882, 1012044761.0439962, 984850183.6360974, 946503741.0634441, 1066338330.4161708, 975532301.7352574, 1014033222.4577638, 21087539.547314797, 20448695.729421042, 18571123.138002872, 24235415.47027013, 15316146.302293217, 13748736.512761574, 17874061.47072198, 17470061.33925307, 17296677.205622133, 19482665.773838963, 24361075.394570667, 18942808.77274743, 21596542.223255727, 23222251.942889407, 22483060.49088308, 18724069.92261987, 20371979.13135991, 16265837.919985846, 20947644.77466151, 6272074.464099809, 20654119.46383206, 15035630.57370

In [35]:
print(naive_obj_list)

[1023637023.7032478, 1094743926.7642622, 1104218878.9781742, 1133157134.0716136, 994298729.3142426, 1164835839.6407163, 1026096352.0176448, 1030844655.4446601, 1095763249.3793414, 1085684091.164103, 988026840.9820616, 1033359141.1945574, 1009810172.4898423, 1145876536.6567209, 1052507022.4175783, 1107975213.109032, 1126451418.7090666, 1074186084.9613397, 1057098901.4817814, 1072639498.4041915, 1242833188.404416, 1183691451.5043302, 1111381007.2226658, 1143308585.93823, 1143512444.447097, 1113474999.754944, 1065174345.5992577, 1198831967.6432836, 1097600226.883317, 1150681015.8129168, 33082032.92770076, 29917247.69684016, 27688935.2134603, 33887428.98596195, 25281555.1652452, 24092115.209496833, 29052583.363034848, 30870699.186881587, 27306643.087623313, 27396049.907603726, 35741455.952813305, 26108758.357063416, 30196135.482895285, 32072116.206327718, 30818608.676351544, 22799992.62899768, 30588516.15113832, 25444999.256008882, 30569709.35653816, 15825556.29792629, 30020105.368613206, 

In [36]:
print(naive_optimality_gap)

[0.12433552648161778, 0.14207580214525314, 0.13422170862337698, 0.14218881809391912, 0.13982561495362683, 0.12755559624345242, 0.13145335051244914, 0.11523720906765324, 0.1250503008766959, 0.1186568426157344, 0.13349975672439354, 0.12766887654352807, 0.12340994495646303, 0.12425841780977231, 0.12408039733224936, 0.1267310435436336, 0.11418223885181619, 0.12667023327973712, 0.11842826080306802, 0.11207271470164462, 0.13792458973097194, 0.12366987084900087, 0.14073289971272557, 0.13017028611149042, 0.12990303241872678, 0.1306034341629098, 0.12537785049056566, 0.12425103126078485, 0.12512955740258688, 0.13475672229352878, 0.5687952998723977, 0.46303940812205524, 0.4909671864056119, 0.3982606994104169, 0.6506472755134175, 0.7523148535965132, 0.6254046910727862, 0.7670630106786709, 0.578721899183476, 0.4061756345679726, 0.4671542768091007, 0.37829393044528153, 0.3981930612197395, 0.38109414561528415, 0.37074793215312235, 0.21768358712727479, 0.5014994838695586, 0.5643214558743755, 0.4593387

In [37]:
print(heuristic_optimality_gap)

[0.008859115522273935, 0.00812058295695038, 0.007817565260117355, 0.00824225006809067, 0.008654211279554925, 0.008065994565251814, 0.00844717989492633, 0.008538940199588733, 0.008065581168960126, 0.00791659011938678, 0.008663745816951689, 0.0078113704834586375, 0.008951649199200077, 0.008637195118302562, 0.008153202574179848, 0.00746822247714581, 0.008049430673669497, 0.008715622025568049, 0.007890951539780859, 0.008952503130727102, 0.007746099194797318, 0.007574605602009454, 0.007469165483114652, 0.0078116634162345875, 0.007628450271075274, 0.008043739909212937, 0.008334247641477952, 0.008113029307063424, 0.008356732127282373, 0.007215351268483157, 0.005639520928935571, 0.0015901517396594111, 0.00329624373712534, 0.006235780657509904, 0.007037856333349989, 0.01037267292295973, 0.00655098846134021, 0.007987326185163526, 0.0008137077880701614, 0.006622225708863589, 0.0066292672179548, 0.009708686475912618, 0.009983797343757459, 0.005891433352176486, 0.005892198608711627, 0.0024445072160

In [38]:
print(len(naive_optimality_gap))

210


In [39]:
print(len(heuristic_optimality_gap))

210


In [44]:
data = {"LR_obj":LR_obj_list,"heuristic_obj":heuristic_obj_list,"naive_obj":naive_obj_list,"LR_time":LR_computation_time_list,"heu_time":heuristic_computation_time_list,"naive_time":naive_computation_time_list,"heu_gap":heuristic_optimality_gap,"naive_gap":naive_optimality_gap}
df = pd.DataFrame(data)
df.sort_values(by=["heu_gap"], ascending=True, inplace=True)
df.head(210)

,LR_obj,heuristic_obj,naive_obj,LR_time,heu_time,naive_time,heu_gap,naive_gap
46,2.037198e+07,2.037730e+07,3.058852e+07,0.014064,0.000405,0.014557,0.000261,0.501499
38,1.729668e+07,1.731075e+07,2.730664e+07,0.016207,0.000408,0.035285,0.000814,0.578722
54,1.619789e+07,1.621847e+07,2.806730e+07,0.015233,0.000385,0.015892,0.001271,0.732775
31,2.044870e+07,2.048121e+07,2.991725e+07,0.046403,0.000478,0.016091,0.001590,0.463039
45,1.872407e+07,1.876984e+07,2.279999e+07,0.017509,0.000411,0.015109,0.002445,0.217684
...,...,...,...,...,...,...,...,...
138,1.040327e+09,1.057999e+09,1.172515e+09,0.231543,0.014204,0.259894,0.016987,0.127063
126,9.241757e+08,9.400281e+08,1.041281e+09,0.230030,0.013232,0.245522,0.017153,0.126713
127,9.693714e+08,9.863855e+08,1.091635e+09,0.192131,0.034762,0.266365,0.017552,0.126127
129,9.039592e+08,9.205481e+08,1.021097e+09,0.249562,0.032191,0.240236,0.018351,0.129583


In [45]:
df[df["heu_gap"] < 0].head(10)

,LR_obj,heuristic_obj,naive_obj,LR_time,heu_time,naive_time,heu_gap,naive_gap


In [41]:
df.to_csv("result.csv",index=False)

## Get both solution for all instances